In [17]:
import random

class FakeNews:
    def __init__(self, propaganda_level, source_credibility, quality_of_writing,
                 virality, target_audience, topic_sensitivity, debunkability, 
                 longevity, social_media_platform ):
        self.propaganda_level = propaganda_level  # Level of persuasion or manipulation
        self.source_credibility = source_credibility  # e.g., big account vs small account
        self.quality_of_writing = quality_of_writing  # Well-written vs poorly written
        self.virality = virality  # Likelihood of spreading based on emotional impact
        self.target_audience = target_audience  # Demographics or group of people targeted
        self.topic_sensitivity = topic_sensitivity  # Emotional or sensitive topic
        self.debunkability = debunkability  # How easy it is to disprove
        self.longevity = longevity  # How long the fake news remains relevant
        self.social_media_platform = social_media_platform  # Platform used to spread

    def get_spreadability_score(self):
        """Calculate the spreadability score of fake news based on its properties."""
        score = 0
        score += self.propaganda_level * 0.2
        score += self.source_credibility * 0.3
        score += self.quality_of_writing * 0.2
        score += self.virality * 0.3
        #score += 
        return score

    def is_dangerous(self):
        """Determine if the fake news is particularly dangerous based on its properties."""
        # Fake news that is well-written, highly viral, and comes from a trusted source is more dangerous
        return self.get_spreadability_score() > 0.75 and self.impact_potential > 0.8

    def evaluate_impact(self):
        """Evaluate the potential impact of fake news based on its characteristics."""
        # Fake news on sensitive topics with high impact potential can have devastating effects
        if self.topic_sensitivity > 0.7 and self.impact_potential > 0.8:
            return "High Impact"
        elif self.topic_sensitivity > 0.5 and self.impact_potential > 0.5:
            return "Moderate Impact"
        else:
            return "Low Impact"

    def check_virality(self):
        """Assess if the fake news will go viral based on its properties."""
        # Fake news with high virality and good quality writing is more likely to spread
        if self.virality > 0.7 and self.quality_of_writing > 0.6:
            return True
        return False

    def debunk_news(self):
        """Assess how easily the fake news can be debunked."""
        if self.debunkability > 0.7:
            return "Easy to debunk"
        elif self.debunkability > 0.4:
            return "Moderately easy to debunk"
        else:
            return "Hard to debunk"


In [14]:
from enum import Enum
class SocialPlatform(Enum):
    LinkedIn = 0
    Facebook = 1
    Instagram = 2
    X = 3
    Telegram = 4
    Reddit = 5
    Pinterest = 6
    Snapchat = 7
    TikTok = 8
    YouTube = 9
    WeChat = 10
    Weibo = 11
    Other = 12
    

In [16]:
from mesa import Agent
class UserAgent(Agent):
    def __init__(
        self, model, initial_state, susceptibility, 
        social_network_size, trust_level, 
        share_propensity, content_evaluation_ability,
        media_consumption_pattern, sentiment_towards_fake_news
    ):
        super().__init__(model)
        
        self.state = initial_state  # State (Susceptible, Exposed, etc.)
        self.susceptibility = susceptibility  # How likely the agent is to believe in fake news
        self.social_network = self.generate_social_network(social_network_size)  # List of friends/followers
        self.trust_level = trust_level  # Trust in social media or news
        self.share_propensity = share_propensity  # Likelihood of sharing news
        self.content_evaluation_ability = content_evaluation_ability  # Ability to judge if news is fake
        self.media_consumption_pattern = media_consumption_pattern  # Which media sources they trust
        self.sentiment_towards_fake_news = sentiment_towards_fake_news  # Reaction to fake news

    def generate_social_network(self, size):
        """Generates a list of social network contacts (other agents)"""
        return random.sample(self.model.schedule.agents, size)
    
    def update(self):
        """Updates the agent's state based on interactions with the network"""
        if self.state == State.Susceptible:
            self.check_for_fake_news()
    
    def check_for_fake_news(self):
        """Check if any of the agent's friends are spreading fake news"""
        for friend in self.social_network:
            if friend.state == State.Infected:  # Infected agents are spreading fake news
                if random.random() < self.susceptibility:  # Probability of agent becoming exposed
                    self.state = State.Exposed
                    break

    def decide_to_share(self, content_type):
        """Decide whether to share a piece of content based on susceptibility, trust, and sentiment"""
        if content_type == "fake_news" and random.random() < self.share_propensity:
            if random.random() < self.trust_level:
                return True
        return False

    def evaluate_content(self, content):
        """Evaluate if the content is fake news or not based on their evaluation ability"""
        if random.random() < self.content_evaluation_ability:
            return False  # Content is judged as real
        else:
            return True  # Content is judged as fake
    
    def consume_media(self):
        """Simulate the agent consuming media based on their pattern and trust level"""
        # Example logic: Based on media consumption patterns, decide if fake news is consumed
        if self.media_consumption_pattern == "social_media":
            # Simulate probability of encountering fake news on social media
            if random.random() < self.sentiment_towards_fake_news:
                # Assume they might be influenced and share fake news
                return self.decide_to_share("fake_news")
        return False
    

In [11]:
from enum import Enum
class State(Enum):
    Susceptible = 0
    Exposed = 1
    Infected = 2
    Doubtful = 3
    Recovered = 4